<a href="https://colab.research.google.com/github/ipeirotis/dealing_with_data/blob/master/01-Pandas/B2a-Inserting_Data_in_MySQL_using_PythonPandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Inserting and Reading data from MySQL using Pandas

First let's start with a basic piece of code that fetches the data that we want to insert in the database. For our example, we will get the data about the Citibike stations, using the correspoding API call provided by the Citibike website:

In [ ]:
!pip install -U -q PyMySQL sqlalchemy sql_magic

In [ ]:
import requests

In [ ]:
# Let's get the data from the Citibike API

# This gives information for each station that remains stable over time
url_stations = "https://gbfs.citibikenyc.com/gbfs/en/station_information.json"
# This gives the live status of all the stations (e.g., bikes available etc)
url_status = "https://gbfs.citibikenyc.com/gbfs/en/station_status.json"

# We fetch for now just the time-invariant data
results = requests.get(url_stations).json()

In [ ]:
# We only need a subset of the data in the JSON returned by the Citibike API, so we keep only what we need
data = results["data"]["stations"]

In [ ]:
len(data)

In [ ]:
import pandas as pd

df = pd.DataFrame(data)
df.head(5)

In [ ]:
# We drop the 'rental methods' columns,
# as they contains multiple values and
# we cannot insert lists in a database cell.
df.drop(
    ["rental_methods", "eightd_station_services", "rental_uris"],
    axis="columns",
    inplace=True,
)
df.head(5)

### Writing a Pandas Dataframe in a MySQL Table

In [ ]:
import sqlalchemy
from sqlalchemy import create_engine

conn_string = "mysql+pymysql://{user}:{password}@{host}/".format(
    host="db.ipeirotis.org", user="student", password="dwdstudent2015"
)

engine = create_engine(conn_string)

Once we have connected successfully, we need to create our database:

In [ ]:
# Query to create a database
# In this example, we will try to create the (existing) database "public"
# But in general, we can give any name to the database
db_name = "public"
create_db_query = (
    f"CREATE DATABASE IF NOT EXISTS {db_name} DEFAULT CHARACTER SET 'utf8'"
)

# Create a database
engine.execute(create_db_query)

# And lets switch to the database
engine.execute(f"USE {db_name}")

In [ ]:
# To avoid conflicts between people writing in the same database, we add a random suffix in the tables
# We only create the variable once while running the notebook
import uuid

if "suffix" not in globals():
    suffix = str(uuid.uuid4())[:8]
print(suffix)

### Create Table and Store Data in Database using the `to_sql` command

Then we create the table where we will store our data. Since we already have the data in a Pandas DataFrame, it is very easy to put the data in a database.

In [ ]:
table_name = f"Stations_{suffix}"
# Create a table
# See http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html for the documentation

# This step is optional, but it is good practice to define explicitly the
# data types before storing things in a database. In many cases, this can be ommitted, though.

create_table_query = f"""CREATE TABLE IF NOT EXISTS {db_name}.{table_name} 
                                (legacy_id int,
                                station_id int,
                                region_id int,
                                external_id varchar(50),
                                lat float,
                                lon float,
                                short_name varchar(10),
                                name varchar(250),
                                station_type varchar(10),
                                capacity int,
                                electric_bike_surcharge_waiver bool,
                                eightd_has_key_dispenser bool,
                                has_kiosk bool,
                                PRIMARY KEY(station_id)
                                )"""
engine.execute(create_table_query)

df.to_sql(
    name=table_name,
    schema=db_name,
    con=engine,
    if_exists="append",
    index=False
)

In [ ]:
# Once we have the data in the table, we also specify a primary key
# If we had FOREIGN KEYS we can add them in the same way
add_key_query = f"ALTER TABLE {table_name} ADD PRIMARY KEY(station_id)"
print(add_key_query)
engine.execute(add_key_query)

### Reading from a SQL Database in Python using the `read_sql` command in Pandas

We can similarly read from the database using Pandas

In [ ]:
query = f"SELECT * FROM {db_name}.{table_name}"
print(query)

In [ ]:
df2 = pd.read_sql(query, con=engine)
df2.head(5)

### Export Data from Database to CSV or Excel

And remember that from Pandas it is also possible to export in other formats, such as Excel of CSV.

In [ ]:
# The necessary library to write in Excel
# !sudo pip3 install -U openpyxl

In [ ]:
df2.to_excel("citibike.xlsx")
df2.to_csv("citibike.csv")

### Cleanup

Finally, let's clean up and delete the table that we created

In [ ]:
drop_table_query = f"DROP TABLE IF EXISTS {db_name}.{table_name}"
print(drop_table_query)
engine.execute(drop_table_query)

### Exercise

The `url_status = 'https://gbfs.citibikenyc.com/gbfs/en/station_status.json'` URL contains the status of the stations. Write code that reads the results from that API call, and then stores the data in a separate table. Add a "foreign key" constraint from the Status table to the Stations table that we created above.